In [1]:
# SSQA error analysis

In [2]:
%load_ext autoreload

In [3]:
autoreload 2

In [ ]:
pos_weight = torch.tensor([2])
v1 = torch.tensor([0])
v2 = torch.tensor([1])
ccriterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss = ccriterion(v1, v2)

In [4]:
from tqdm import tqdm_notebook as tqdm
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
import os
from collections import defaultdict

from dataset import * 
from model import * 
import config

I0709 05:19:13.821080 140160478742336 file_utils.py:39] PyTorch version 1.5.1+cu101 available.


In [5]:
model = BertSERModel()

unkown bert model choice, init with PRETRAINED_MODEL_NAME


I0709 05:19:15.750962 140160478742336 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.f12a4f986e43d8b328f5b067a641064d67b91597567a06c7b122d1ca7dfd9741
I0709 05:19:15.753451 140160478742336 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_he

In [6]:
model.load_state_dict(torch.load('../param/baseline/model_epoch1_precision:0.496_recall:0.612_f1:0.492_acc:0.912.m'))

<All keys matched successfully>

In [33]:
model

BertSERModel(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [7]:
model.cpu()
None

In [8]:
dev_set = SSQA_Dataset(config.SSQA_DEV, "develop", None)

I0709 05:19:24.551474 140160478742336 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [17]:
dev_loader = DataLoader(dev_set, batch_size=9, collate_fn=create_mini_batch)

In [32]:
for batch in dev_loader:
    res = model._predict(batch)
    print(res)

[1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 0, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 1, 1

In [29]:
next(iter(dev_loader))

(tensor([[ 101, 6851, 6882,  ..., 1240,  511,  102],
         [ 101, 6851, 6882,  ...,    0,    0,    0],
         [ 101, 6851, 6882,  ...,    0,    0,    0],
         ...,
         [ 101, 6851, 6882,  ...,    0,    0,    0],
         [ 101, 6851, 6882,  ...,    0,    0,    0],
         [ 101, 6851, 6882,  ...,    0,    0,    0]]),
 tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([ True, False, False, False, False, False, False, False, False]))